In [3]:
# Importing the libraries 
import pandas as pd
import numpy as np
import random
import pickle
import re
import regex  # for better, more capbale regex api
import os
import zipfile
import more_itertools
from itertools import chain
import datetime
import time
from statsmodels.stats.proportion import proportion_confint
# active labeler related
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import ComplementNB  # corrects for class imbalance, SGD is pretty good too
from sklearn.pipeline import Pipeline
from superintendent import ClassLabeller
from IPython.display import display, Markdown

pd.set_option('display.max_colwidth', None)  # so we can peak at data and spot verify

pipeline = Pipeline([
    ('vect', CountVectorizer(analyzer='char', ngram_range=(1,2))),
    
    ('tfidf', TfidfTransformer()),
    ('clf', ComplementNB()),
])
print('done')

done


In [4]:
# Set up columns to keep, fields, locations for writing
rootpath = "/hdd/work/d4ad_standardization/"
processedpath = "D4AD_Standardization/data/processed/"
externalpath = "D4AD_Standardization/data/external/"
interimpath = "D4AD_Standardization/data/interim/"

content_is = "standardized_descriptions_and_degree_funding_type"

print('done')

done


In [5]:

filepath = content_is + ".csv" # builds off of notebook 6.0 work

SKIP_THIS = True # helps me be able to run all and not worry about pulling things
# I already know I have on disk

df = pd.read_csv(rootpath + interimpath + filepath)
print('done2')

done2


In [6]:
pd.set_option('display.max_rows', False)

the_df = df #df.sample(n=10000, random_state=42)

In [18]:
training_context_regex =\
    """
    (job[\s\b.].*?search                # job search, mentioned in an order
     |assist[\w\s\b\.].*?employ         # or assitance gaining employment
     |employ[\w\s\b\.].*?assist)        # or employment assistance :)
    """


training_context_regex =\
    """
    ((\w+\W+){0,8}        # first 8 or so words before
     (?P<job_search>job[\s\b.].*?search|assist[\w\s\b\.].*?employ\w*?\b) # help w/ job search
     (\W+\w+){0,8})       # and last 8 or so word after
    """

wioa_indices = the_df['IS_WIOA'] == True
the_df['minimum_six_months_search'] =\
    False
the_df.loc[wioa_indices, 'minimum_six_months_search'] =\
    True

# Extract any possible nearby time, date information and see if
# numbers, units of time are mentioned (e.g. four months)
print(training_context_regex)
job_search_length_mention =\
    the_df.loc[wioa_indices, 'DESCRIPTION']\
          .str\
          .extractall(pat=training_context_regex, flags=re.I|re.VERBOSE)

the_df['additional_search_mention'] =\
    None

job_search_length_mention # wow there's only 33 mentions in all 24k rows
# this might have been a waste of time?


    ((\w+\W+){0,8}        # first 8 or so words before
     (?P<job_search>job[\.].*?search|assist[\w\\.].*?employ\w*) # help w/ job search
     (\W+\w+){0,8})       # and last 8 or so word after
    


internet use and safety. Skills necessary to use job search engines and to apply for and track employment   
3587  0                                                                                                                                                                                                                Four week job search and job readiness assistance means the act of   
5992  0                                                                                                                                                                 spend another three hours per day on individual job search.   Classes will focus on enhancing students job skills   
6025  0                                                                                                                                                                                                                Four week job search and job readiness assistance means the act of   
7463  0                                                                                                                                                                                                 Structured job search and post-job search Work First New Jersey core and non-core   
9800  0                                                                                                                                                              and evaluation.\nIndividual and/or group sessions on job search, resume writing and interviewing skills.\nJVS is non   
9998  0                                                                                                                                                                     service and an office simulation experience.  During the Job Search phase, students learn how to get and keep   
      1                                                                        a job while preparing applications, resumes, cover and thank you letters.  Interviewing strategies will be covered as students begin supported job search where obtaining full-time employment is the goal   
...                                                                                                                                                                                                                                                                                   ...   
19534 0                                                                                                                             Skill Training, Adult Mentoring, Leadership Development, Occupational Training, Job Search Skill Training and Job Development and Supportive Services   
20566 0                                                                                      personal and professional development, lab study/application time, job simlulation/internship, getting and keeping a job, and job search skills.  Students have the opportunity to earn nine   
20600 0                                                                                                                                                              assist the participants with the development of basic job search techniques and acquisition of soft skills needed to   
      1      and secure employment. The Program will going to job interview and coaching will concentrate on common barriers to employment, such as work readiness/social adjustment skills, transportation problems, job search, and maintenance skills. Our computer labs are connected   
      2                          Internet to enable clients to access America¿s Job Bank, WNPIN, etc.  The Program will deliver forty hours of programming over a four-week period, including computer literacy instruction, job search and readiness training, and life skills workshops   
21178 0                                                                                                                                                                                   

In [18]:
# 4)
# Now we write out the verfiied results
# ... finally we can write this out as our first complete lookup table
# for the NAME field
write_out = draft_output

print(
    "We're writing ...",
    write_out.columns
)

content_is = "with_job_search_durations"

# shuffe the rows to better remove temporal baises
write_out =\
    write_out.sample(frac=1, random_state=42, axis=0).reset_index(drop=True)

write_out.to_csv(rootpath + interimpath + content_is + ".csv",
                index = False,
                chunksize = 10000)

write_out.to_excel(rootpath + processedpath + content_is + ".xls",
            sheet_name="Standardized Descriptions",
            index=False)
print('done')

We're writing ... Index(['IS_WOIA', 'Mentioned_Certificate', 'Mentioned_Associates',
       'STANDARDIZED_DESCRIPTION', 'STANDARDIZED_FEATURESDESCRIPTION',
       'CLEANED_STANDARDIZED_NAME_1', 'STANDARDIZEDNAME', 'STANDARDIZEDNAME_1',
       'DESCRIPTION', 'FEATURESDESCRIPTION', 'NAME_1', 'NAME'],
      dtype='object')
done
